سلام دوستان

در کلاس تلاش کردیم از Tensorboard برای ترسیم نمودار خطای آموزش استفاده کنیم. متاسفانه جواب نداد. من موضوع را بررسی کردم. بطور کلی در پایتورچ به دو روش می توانید از tensorboard استفاده کنید. یکی بسته TensorboardX و دیگری ماژول torch.utils.tensorboard. دومی درون پایتورچ از نسخه 1.4 به بعد قرارداده شده است و در 
حقیقت مبتنی بر همان بسته ی Tensorboard که در Tensorflow است کار می کند.

پایتورچی که روی سیستم من بود نسخه 1.1 بود و خیلی قدیمی بود و احتمالا به همین دلیل TensorboardX کار نکرد. اگر به لینک TensorboardX در 
https://github.com/lanpa/tensorboardX
مراجعه کنید میبینید که بسته را روی پایتورچ 1.8 امتحان کرده‌اند.

نهایتا من سیستم خودم را بروز رسانی کردم و اینبار از ماژول torch.utils.tensorboard استفاده کردم.
اگر روی سیستم شما این ماژول کار نکرد از دستور 
pip install tensorboard
استفاده کنید برای نصب tensorboard
همانطور که گفتم این همان تنسوربورد موجود در Tensorflow است که پایتورچ از نسخه 1.4 به بعد می‌تواند آنرا استفاده کند.

برای اطلاعات بیشتر به این لینک مراجعه کنید
https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html


In [1]:
import numpy as np

import torch
import torch.nn as nn
from torch import optim
from torchvision import datasets, transforms


import matplotlib.pyplot as plt

#from tensorboardX import SummaryWriter
# writer = SummaryWriter(log_dir='./TensorboardX_logs/') 


# ====================================================
import datetime
from torch.utils.tensorboard import SummaryWriter

# For each run, we want to create a separate folder to store Tensorboard's content
# In this way, we can easily switch between them in Tensorboard's GUI.
log_folder_name = 'logs/experiment-at-%s'%datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(log_dir = log_folder_name)
# ====================================================

# prepare dataset

In [2]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
#transform = transforms.ToTensor() # PIL image [0, 255] int === ToTensor() ==> PyTorch Tensor [0, 1] float

# Download and load the training data
trainset = datasets.MNIST(root = '~/.pytorch/MNIST_data/', download=True, train=True, transform=transform) # target_transform
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Define the model

In [3]:
model = nn.Sequential(nn.Linear(784, 128),
                     nn.ReLU(),
                     nn.Linear(128, 64),
                     nn.ReLU(),
                     nn.Linear(64, 10),
                     nn.LogSoftmax(dim=1)).cuda() 

In [4]:
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)


In [5]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss() # Negative log-likehood

$w_{new} = w_{old} - \eta*\text{Grad}_{w}(\text{one sample}, \text{criterion})$

In [6]:
learning_rate = 0.003
optimizer = optim.SGD(model.parameters(), lr = learning_rate) 

# Train the model

In [7]:
my_device = 'cuda'

epochs = 5

for epoch in range(epochs):
    
    running_loss = 0
    
    for images, labels in trainloader:
        # Flatten - N*C*H*W ===> N*(C*H*W) <=> 64*784
        images = images.view(images.shape[0], -1)
        
        # Clear the gradients
        optimizer.zero_grad()
        
        # forward computation
        output = model(images.to(my_device))
        
        loss = criterion(output, labels.to(my_device))
        
        
        # backward pass
        loss.backward()
        
        # update
        optimizer.step()
        
        running_loss += loss.item()
    else:
        current_loss = running_loss/len(trainloader)
        print(f"Training loss = {current_loss}")
        # ====================================================
        writer.add_scalar('trainloss', current_loss,epoch)
        # ====================================================

# ====================================================
# It is good to run one of the following two commands:
# writer.flush() # to make sure that all pending events have been written to disk.
writer.close()   # If you do not need the summary writer anymore, call close() method.
# ====================================================

# To run tensorboard, open your command line
# Change your directory to where that this directory (Tensorboard_logs) can be accessed with the following command
# Run this on your command line
# tensorboard --logdir=Tensorboard_logs
# Goto the address given by tensorboard

Training loss = 2.251042738398001
Training loss = 1.9874214423236563
Training loss = 1.33196158087584
Training loss = 0.8309394008061016
Training loss = 0.6249460220845269


After training, when you run the tensorboard, you should observe something similar to the following figure

<img src='tensorboard_example_screenshot.png'></img>

# Prediction / Inference

In [8]:
images, labels = next(iter(trainloader))

In [9]:
data_index = 0
images[data_index].shape  # torch.Size([1, 28, 28])

input_vector = images[data_index].view(1, 784)

# Turn off gradient computation
with torch.no_grad():
    scores = model(input_vector.to(my_device))

print(scores) # logits

tensor([[-7.5409, -9.2886, -4.3122, -7.2183, -0.2336, -6.1455, -5.8437, -3.9721,
         -4.2623, -1.8606]], device='cuda:0')


In [10]:
print(nn.Softmax(dim=1)(scores))

import torch.nn.functional as F
print(F.softmax(scores))

tensor([[5.3091e-04, 9.2474e-05, 1.3404e-02, 7.3304e-04, 7.9170e-01, 2.1432e-03,
         2.8982e-03, 1.8833e-02, 1.4091e-02, 1.5558e-01]], device='cuda:0')
tensor([[5.3091e-04, 9.2474e-05, 1.3404e-02, 7.3304e-04, 7.9170e-01, 2.1432e-03,
         2.8982e-03, 1.8833e-02, 1.4091e-02, 1.5558e-01]], device='cuda:0')


C:\Users\ashkan\AppData\Local\Temp\ipykernel_6720\1124326387.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(scores))


In [11]:
print(torch.argmax(nn.Softmax(dim=1)(scores)))

tensor(4, device='cuda:0')


In [12]:
print(torch.topk(nn.Softmax(dim=1)(scores), 1))

torch.return_types.topk(
values=tensor([[0.7917]], device='cuda:0'),
indices=tensor([[4]], device='cuda:0'))


In [13]:
labels

tensor([4, 6, 0, 5, 4, 5, 6, 7, 6, 7, 7, 7, 8, 1, 5, 5, 0, 3, 1, 5, 4, 0, 6, 0,
        3, 7, 6, 5, 3, 2, 2, 1, 1, 4, 0, 9, 5, 4, 6, 6, 3, 8, 1, 5, 0, 4, 3, 9,
        4, 5, 4, 0, 5, 6, 7, 5, 4, 3, 4, 9, 6, 0, 4, 0])

In [14]:
labels[0]

tensor(4)